In [1]:
import json
import hashlib
from hashlib import sha256
import time

In [2]:
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

In [3]:
class Blockchain: 
    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.difficulty = 4
        self.create_genesis_block()
 
    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
    @property
    def last_block(self):
        return self.chain[-1]

    def proof_of_work(self, block):
        block.nonce = 1
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash    

    def add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
 
    def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * self.difficulty) and
                block_hash == block.compute_hash())

    def add_new_transaction(self, transaction):
            self.unconfirmed_transactions.append(transaction)
 
    def mine(self):
        if not self.unconfirmed_transactions:
            return False
 
        last_block = self.last_block
 
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
 
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = []
        return new_block.index

In [4]:
tx_1 = {
   "addr_from": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 99.00
}

In [5]:
tx_2 = {
   "addr_from": "1J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "2BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 199.00
}

In [6]:
Titans = Blockchain()
while len(Titans.chain)<10:
  Titans.add_new_transaction([tx_1])
  Titans.mine()
  print(Titans.chain[-1].hash)
  print(str(Titans.chain[-1].nonce))


000068c8ae993b4340e577b4f3b44bef3d12b4f5f73372621e57232980cca317
2603
000011a0f93544d9e6fe10b90730d96851c66938e38a032f03044f81168478e0
43062
0000b7337cfe7623aeb033bc970d5ce440a4a7947f248ed9765d4262b81936bf
278417
00008e0c562b0fe3a2341cb24bfe4ff2c65756186ebb80e8b26a6ffa5b84543f
104210
000085e427bb799cd13aaae3df8ab98115e2d6d0833944d2f3ad6c2f84f773a1
6517
0000103da3762c77b88a0c128636b56142914f308d162d125a7dd5fa1c9b41a4
74342
0000f65dae5fcbad4108449cb526384fbdb977280348fe76860d87639d013d8f
151562
0000c892b44bb6989aa45232439aacfd97fca7693220651a9e9e099b609a1c7c
35071
00007d7fa9a92364e79e70c3265b58296dfe5f96e9cfeb7d1094edc17e8e61a7
106353
